In [21]:
#
# This example is for the implementation of TimeDistributed / LSTM layers in MUSCo
#

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tqdm import tqdm
from glob import glob
import importlib
from IPython.core.debugger import set_trace

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [3]:
import musco
from musco.tf.compressor.compress import compress_seq, compress_noseq

In [4]:
def prepare_data(batch_size=20, test_batch_size=1000):
    # prepare mnist dataset
    num_class = 10

    # Load and preprocess the dataset
    # data shape is x: [?, n_rows, n_columns], y: [?, 1]
    (x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

    # data shape is x: [n_examples, n_rows, n_columns], y: [?, n_class]
    # train_ds is infinitely repeated
    train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    train_ds = train_ds.map(lambda x, y: (x / 255,
                      tf.one_hot(y, depth=num_class)))
    train_ds = train_ds.shuffle(buffer_size=1024).batch(batch_size)

    test_ds =  tf.data.Dataset.from_tensor_slices((x_test, y_test))
    test_ds = test_ds.map(lambda x, y: (x / 255,
                      tf.one_hot(y, depth=num_class)))
    test_ds = test_ds.batch(test_batch_size)
    return train_ds, test_ds

In [5]:
def get_td_model():
    input_x = layers.Input(shape=(28, 28))
    out = layers.Reshape([7, 112])(input_x)
    out = layers.TimeDistributed(layers.Dense(32, activation='relu'), name='td_0')(out)
    out = layers.Flatten()(out)
    out = layers.Dense(10, name='dense0', activation='sigmoid')(out)
    return keras.Model(inputs=input_x, outputs=out)


In [6]:
def get_lstm_model():
    input_x = layers.Input(shape=(28, 28))
    out = layers.Reshape([7, 112])(input_x)
    out = layers.LSTM(
        32, return_sequences=False, name='lstm_0')(out)
    out = layers.Dense(10, name='dense0', activation='sigmoid')(out)
    return keras.Model(inputs=input_x, outputs=out)


In [7]:
def get_functional_model():
    input_x = layers.Input(shape=(28, 28))
    out = layers.Flatten()(input_x)
    bypass = out
    out = layers.Dense(32, activation='relu', name='fc_0')(out)
    out = layers.Concatenate()([out, bypass])
    out = layers.Dense(10, activation='sigmoid', name='fc_1')(out)
    return keras.Model(inputs=input_x, outputs=out)


In [8]:
def train_model(model, train_ds, test_ds, epochs=5, verbose=0):
    model.compile(loss='categorical_crossentropy', metrics=['categorical_accuracy'])
    hist = model.fit(train_ds, epochs=epochs, validation_data=test_ds, verbose=verbose)
    accuracy_val = hist.history['val_categorical_accuracy'][-1]
    loss_val = hist.history['val_loss'][-1]
    return loss_val, accuracy_val


In [9]:
model = get_functional_model()
train_ds, test_ds = prepare_data(batch_size=64, test_batch_size=1024)
train_model(model, train_ds, test_ds, verbose=1)

(0.1614006906747818, 0.9537000060081482)

In [15]:
#
# Tests which MUSCo should eventually pass
#

In [18]:
%pdb off

Automatic pdb calling has been turned OFF


In [15]:
# TimeDistributed
model = get_td_model()
train_ds, test_ds = prepare_data(batch_size=64, test_batch_size=1024)
model.compile(loss='categorical_crossentropy', metrics=['categorical_accuracy'])
decompose_info = {'td_0': ('svd', 10)}
model_compress = compress_seq(model, decompose_info)
train_model(model_compress, train_ds, test_ds, epochs=2, verbose=1)

100%|██████████| 5/5 [00:00<00:00, 30.50it/s]


Epoch 1/2
938/938 [==============================] - 6s 6ms/step - loss: 0.5239 - categorical_accuracy: 0.8527 - val_loss: 0.3405 - val_categorical_accuracy: 0.9005
Epoch 2/2
938/938 [==============================] - 6s 6ms/step - loss: 0.3288 - categorical_accuracy: 0.9058 - val_loss: 0.3163 - val_categorical_accuracy: 0.9101


(0.31628841161727905, 0.910099983215332)

In [19]:
# LSTM
model = get_lstm_model()
train_ds, test_ds = prepare_data(batch_size=64, test_batch_size=1024)
model.compile(loss='categorical_crossentropy', metrics=['categorical_accuracy'])
decompose_info = {'lstm_0': ('svd', 10)}
model_compress = compress_seq(model, decompose_info)
train_model(model_compress, train_ds, test_ds, epochs=1, verbose=1)

100%|██████████| 4/4 [00:00<00:00, 14.33it/s]


938/938 [==============================] - 11s 12ms/step - loss: 1.7097 - categorical_accuracy: 0.4415 - val_loss: 1.1080 - val_categorical_accuracy: 0.6591


(1.1079920530319214, 0.6590999960899353)

In [10]:
# Non-sequential models
model = get_functional_model()
train_ds, test_ds = prepare_data(batch_size=64, test_batch_size=1024)
model.compile(loss='categorical_crossentropy', metrics=['categorical_accuracy'])
decompose_info = {'fc_0': ('svd', 10)}
# set_trace()
model_compress = compress_noseq(model, decompose_info)
train_model(model_compress, train_ds, test_ds, epochs=1, verbose=1)

{Reconstruction}: 100%|██████████| 4/4 [00:00<00:00, 134.20it/s]


938/938 [==============================] - 5s 5ms/step - loss: 0.4484 - categorical_accuracy: 0.8824 - val_loss: 0.3031 - val_categorical_accuracy: 0.9131


(0.30307307839393616, 0.913100004196167)